In [ ]:
import os
from os.path import join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob
import shutil
from tqdm.notebook import tqdm

In [ ]:
state = "punjab"

txt_path = f"/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/yolo_tp/{state}_tp/unprocessed_labels"
fcbk_path = f"/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/yolo_tp/{state}_tp/fcbk_tp_unprocessed"
zigzag_path = f"/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/yolo_tp/{state}_tp/zigzag_tp_unprocessed"
new_txt_path = "/tmp/corrected_txt"

def process_image(path, class_name):
    file_name = path.split("/")[-1].rsplit(".", 1)[0]
    
    lat, lon = file_name.split(",")
    lon_precision = lon.split(".")[1]
    if len(lon_precision) == 2:
        txt_file_name = file_name
        line_number = 0
    elif len(lon_precision) == 3:
        txt_file_name = file_name[:-1]
        line_number = int(lon_precision[-1]) - 1
    else:
        raise ValueError("Invalid longitude precision")
    
    with open(join(txt_path, f"{txt_file_name}.txt"), "r") as f:
        lines = f.readlines()
        
    line_of_focus = lines[line_number]
    elements = line_of_focus.split()
    elements_no_conf = elements[:-1]
    
    # process
    if class_name == "zigzag":
        elements_no_conf[0] = "0"
    elif class_name == "fcbk":
        elements_no_conf[0] = "1"
    else:
        raise ValueError("Invalid class name")
    
    new_line = " ".join(elements_no_conf)
        
    with open(join(save_dir, f"{txt_file_name}.txt"), "a") as f:
        print(new_line, file=f)    

txt_files = glob(join(txt_path, "*.txt"))
image_files = {"fcbk": glob(join(fcbk_path, "*.jpg")), "zigzag": glob(join(zigzag_path, "*.jpg"))}
save_dir = join(new_txt_path, state)
shutil.rmtree(save_dir, ignore_errors=True)
os.makedirs(save_dir)

for class_name in image_files:
    class_image_files = image_files[class_name]
    for path in tqdm(class_image_files):
        process_image(path, class_name)
    